<a href="https://colab.research.google.com/github/12Siva/story-weaver-engine/blob/main/StoryWeaver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# StoryWeaver: A Python script adapted for Google Colab.

# STEP 1: Install all necessary packages
# Run this cell first to install the required system and Python libraries.
!apt-get -qq install graphviz
!pip install -q google-generativeai graphviz pypdf reportlab

# STEP 2: Import libraries and set up your API Key
import os
import json
import re
import io  # Needed to handle the uploaded file in memory
import pypdf # The library for reading PDF text

import google.generativeai as genai
from graphviz import Digraph
from google.colab import userdata
from google.colab import files # The library for handling file uploads in Colab
from IPython.display import Image, display

# Access your Gemini API key from Colab's secrets manager
try:
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("Successfully configured Gemini API key.")
except userdata.SecretNotFoundError:
    print("ERROR: Secret 'GEMINI_API_KEY' not found.")
    print("Please add it to the Colab Secrets manager (key icon on the left).")
except Exception as e:
    print(f"An error occurred during API key configuration: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.2 MB/s eta 0:00:00
Successfully configured Gemini API key.


In [2]:
# STEP 3: Define the StoryWeaver functions

def upload_and_extract_pdf_text() -> str | None:
    """
    Displays a file upload button in Colab, allows user to upload a PDF,
    and extracts all text from it.
    """
    print("Please upload a PDF file...")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded. Aborting.")
        return None

    # Get the filename of the first uploaded file
    file_name = next(iter(uploaded))
    print(f"Processing uploaded file: {file_name}")

    try:
        # Use io.BytesIO to treat the uploaded bytes as a file
        pdf_file = io.BytesIO(uploaded[file_name])
        reader = pypdf.PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""

        if not text.strip():
            print("Warning: Could not extract any text from the PDF. It might be an image-based PDF.")
            return None

        print("Successfully extracted text from PDF.")
        return text
    except Exception as e:
        print(f"An error occurred while reading the PDF: {e}")
        return None

def get_structured_flowchart_data(story_text: str) -> dict:
    """
    Uses the Gemini LLM to process a story and generate a structured
    JSON representation of its flowchart.
    """
    prompt = f"""
    System Instruction:
    You are an expert system designed to analyze children's stories and convert them into a structured flowchart format.
    Your sole output must be a single, valid JSON object that represents the story's flow, focusing on the main characters' actions and decisions.
    Do not include any explanatory text or markdown formatting around the JSON.

    The JSON object must follow this schema:
    - "diagram_type": A string, which should be "flowchart".
    - "nodes": An array of objects, where each object has:
      - "id": A unique, simple string identifier for the node (e.g., "node1", "node2").
      - "label": A short, descriptive text for the step or event in the story.
      - "shape": A string defining the node's shape. Use "box" for actions/events and "diamond" for decisions/questions.
    - "edges": An array of objects, where each object represents a connection and has:
      - "from": The "id" of the source node.
      - "to": The "id" of the destination node.
      - "label": (Optional) A string for labelling the connection, especially for decisions (e.g., "Yes", "No", "Follows butterfly").

    ---
    Few-Shot Example:

    Story Input:
    "A cat sat on a mat. It saw a mouse. Should it chase the mouse? The cat decided yes and chased the mouse into a hole."

    Expected JSON Output:
    {{
      "diagram_type": "flowchart",
      "nodes": [
        {{"id": "node1", "label": "A cat sits on a mat", "shape": "box"}},
        {{"id": "node2", "label": "It sees a mouse", "shape": "box"}},
        {{"id": "node3", "label": "Chase the mouse?", "shape": "diamond"}},
        {{"id": "node4", "label": "Chase mouse into a hole", "shape": "box"}}
      ],
      "edges": [
        {{"from": "node1", "to": "node2", "label": ""}},
        {{"from": "node2", "to": "node3", "label": ""}},
        {{"from": "node3", "to": "node4", "label": "Yes"}}
      ]
    }}
    ---

    Story Input:
    "{story_text}"

    Expected JSON Output:
    """

    print("Sending story to Gemini for analysis...")
    try:
        model = genai.GenerativeModel('gemini-2.5-pro')
        response = model.generate_content(prompt)
        json_text = re.sub(r'```json\n?|```', '', response.text).strip()
        print("Received structured data from Gemini. Parsing JSON...")
        return json.loads(json_text)
    except Exception as e:
        print(f"An error occurred while calling the Gemini API or parsing the JSON: {e}")
        if 'response' in locals():
            print("\n--- Raw Response Text ---\n", response.text, "\n--------------------------")
        return None

def render_flowchart_for_colab(flowchart_data: dict, output_filename: str):
    """
    Takes structured data, saves it as a PNG file using Graphviz,
    and returns the filename.
    """
    if not flowchart_data:
        print("Cannot render flowchart, input data is empty.")
        return None

    print("Rendering flowchart...")
    dot = Digraph(comment='StoryWeaver Flowchart')
    dot.attr('node', shape='box', style='rounded,filled', fillcolor='skyblue', fontname='Arial')
    dot.attr('edge', color='gray40', fontname='Arial', fontsize='10')
    dot.attr(rankdir='TB', splines='ortho')

    for node in flowchart_data.get("nodes", []):
        dot.node(name=node["id"], label=node["label"], shape=node.get("shape", "box"))

    for edge in flowchart_data.get("edges", []):
        dot.edge(tail_name=edge["from"], head_name=edge["to"], label=edge.get("label", ""))

    try:
        dot.render(output_filename, format='png', cleanup=True)
        image_filename = f"{output_filename}.png"
        print(f"Successfully rendered flowchart to {image_filename}")
        return image_filename
    except Exception as e:
        print(f"An error occurred during rendering: {e}")
        return None


In [10]:
def rewrite_story(original_story: str, user_request: str) -> str | None:
    """
    Uses Gemini to rewrite the story based on a user's request.
    """
    print("Sending request to Gemini to rewrite the story...")

    prompt = f"""
    You are a creative storyteller for children.
    Your task is to rewrite the following story, but incorporate the user's requested change.
    Make the new ending creative, fun, and consistent with the tone of the story.
    Your response must ONLY be the complete, new version of the story. Do not include any introductory or concluding remarks.

    Original Story:
    ---
    {original_story}
    ---

    User's Requested Change:
    ---
    "{user_request}"
    ---

    Now, provide the complete, new version of the story.
    """

    try:
        model = genai.GenerativeModel('gemini-2.5-pro')
        response = model.generate_content(prompt)
        print("Received new story from Gemini.")
        return response.text
    except Exception as e:
        print(f"An error occurred while rewriting the story: {e}")
        return None

In [11]:
# STEP 4: Save the final story as a PDF

def save_story_as_pdf(story_text: str, output_filename: str = "branched_story.pdf"):
    """
    Saves the given story text as a PDF file with improved formatting.
    """
    try:
        from reportlab.lib.pagesizes import letter
        from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
        from reportlab.lib.styles import getSampleStyleSheet

        doc = SimpleDocTemplate(output_filename, pagesize=letter)
        styles = getSampleStyleSheet()
        story = []

        # Split the text into paragraphs based on double newlines
        paragraphs = story_text.split('\n\n')

        for para_text in paragraphs:
            if para_text.strip(): # Avoid adding empty paragraphs
                story.append(Paragraph(para_text.strip(), styles["Normal"]))
                story.append(Spacer(1, 0.1 * letter[1])) # Add some space after each paragraph

        doc.build(story)
        print(f"Story successfully saved to {output_filename}")
    except ImportError:
        print("ReportLab library not found. Please install it: !pip install reportlab")
    except Exception as e:
        print(f"An error occurred while saving the PDF: {e}")

# Assuming 'story_to_process' holds the final story text from the previous cell
if 'story_to_process' in locals() and story_to_process:
    pdf_output_filename = "final_story.pdf"
    save_story_as_pdf(story_to_process, pdf_output_filename)
else:
    print("No story text available to save as PDF.")

Story successfully saved to final_story.pdf


In [4]:
# Sample story and run the process
# To bypass reading from a local file.
story_to_weave = """
Once upon a time, in a green meadow, lived a curious rabbit named Barnaby. One sunny morning, Barnaby saw a beautiful butterfly with shimmering wings. "I must follow it!" he thought.

The butterfly led Barnaby through a field of flowers where he met a wise old tortoise named Sheldon. "Where are you going in such a hurry?" asked Sheldon. Barnaby explained he was following the magical butterfly.

Sheldon smiled and pointed with his head. "It flew towards the Whispering Woods. But be careful, a grumpy fox named Mr. Fox lives there." Barnaby, being brave, thanked Sheldon and hopped towards the woods.

Inside the woods, the trees whispered secrets. Barnaby soon stumbled upon Mr. Fox, who was napping. The butterfly landed gently on Mr. Fox's nose, waking him up with a twitch. Mr. Fox sneezed, and the butterfly flew away. Barnaby laughed and Mr. Fox, surprised, couldn't help but chuckle too. From that day on, Barnaby and Mr. Fox became unlikely friends.
"""

In [ ]:
def display_story_as_flowchart(story_text: str):
    # Generate the flowchart from the (potentially new) story
    structured_data = get_structured_flowchart_data(story_to_process)
    if structured_data:
        output_filename_base = "storyweaver_flowchart"
        image_file = render_flowchart_for_colab(structured_data, output_filename_base)

        if image_file:
            print(f"Displaying image: {image_file}")
            display(Image(filename=image_file))

In [ ]:
# --- Main Execution (Now with Interactive Ending) ---

# First, get the original story from the PDF
original_story = upload_and_extract_pdf_text()

# Only proceed if we have a story
if original_story:
    # Ask the user if they want to change the ending
    ending_request = input("Enter a new ending or a change you'd like to see (or press Enter to skip): ")

    story_to_process = original_story

    # If the user entered a request, rewrite the story
    if ending_request.strip():
        new_story = rewrite_story(original_story, ending_request)
        if new_story:
            story_to_process = new_story
            save_story_as_pdf(story_to_process, "new_story.pdf")
    else:
        print("No changes requested. Processing the original story.")

